# COMP 6934 - Assignment 6

Name: Mohammad Shehabul Islam

ID: 202196528

# Question 1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [2]:
df_diamond = pd.read_csv('diamonds.csv')
df_diamond.head(3)

,id,url,shape,price,carat,cut,colour,clarity,report,type,date_fetched
0,10086429,https://www.brilliantearth.com//loose-diamonds...,Round,400,0.30,Very Good,J,SI2,GIA,natural,2020-11-29 12-26 PM
1,10016334,https://www.brilliantearth.com//loose-diamonds...,Emerald,400,0.31,Ideal,I,SI1,GIA,natural,2020-11-29 12-26 PM
2,9947216,https://www.brilliantearth.com//loose-diamonds...,Emerald,400,0.30,Ideal,I,VS2,GIA,natural,2020-11-29 12-26 PM


In [3]:
df_diamond = pd.read_csv('diamonds.csv')

def plot_figure(Origin):
    if Origin == 'Natural':
        df=df_diamond[(df_diamond.type == 'natural')]
    elif Origin == 'Lab':
        df=df_diamond[(df_diamond.type == 'lab')]
    else:
        df=df_diamond
    df_hm = df[['colour', 'clarity', 'carat']].groupby(['clarity', 'colour']).mean().reset_index()
    df_hm = df_hm.pivot(index='colour', columns='clarity', values='carat')
    desired_order = ['SI2','SI1','VS2','VS1','VVS2','VVS1','IF']
    df_pivot = df_hm.reindex(columns=desired_order)
    sns.color_palette('mako')
    sns.heatmap(data=df_pivot, cmap="mako")
interact(plot_figure, Origin=['All','Natural','Lab'])

interactive(children=(Dropdown(description='Origin', options=('All', 'Natural', 'Lab'), value='All'), Output()…

<function __main__.plot_figure(Origin)>

# Question 2

In [4]:
data = pd.read_csv("olympic_athletes.csv")
data.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [5]:
data = pd.read_csv("olympic_athletes.csv")

gld_medalists = data[(data['Medal'] == 'Gold') & (data['Year'] >= 2000) & (data['Year'] <= 2016)]
gld_medalists = gld_medalists.dropna(subset=['Height', 'Weight'])

def plot_figure(sport, kde, display_data):
    if sport != 'Combined':
        filtered_data = gld_medalists[gld_medalists['Sport'] == sport]
    else:
        filtered_data = gld_medalists  
    plt.clf()
    
    fig, ax = plt.subplots(figsize=(7, 5))
    plt.xlim(20, 140)
    plt.ylim(130, 220)
    plt.xlabel('Weight')
    plt.ylabel('Height')

    if not kde:
        weight_bins = np.linspace(30, 140, 30)  
        height_bins = np.linspace(135, 220, 30)  
        counts, _, _ = np.histogram2d(filtered_data['Weight'], filtered_data['Height'], bins=(weight_bins, height_bins))        
        mesh = ax.pcolormesh(weight_bins, height_bins, counts.T, cmap="Blues", norm=mcolors.LogNorm())
        
    else:
        sns.kdeplot(data=filtered_data, x='Weight', y='Height', fill=True, ax=ax, alpha=0.5)
    
    if display_data:
        male = filtered_data[filtered_data['Sex'] == 'M']
        female = filtered_data[filtered_data['Sex'] == 'F']
        plt.scatter(male['Weight'], male['Height'], color='#047CCB', linewidth=0.5, edgecolor='white', label='M', s=25)
        plt.scatter(female['Weight'], female['Height'], color='#F68D25', linewidth=0.5, edgecolor='white', label='F', s=25)
        plt.legend(title='Sex', loc='upper right')
    

sports = ['Swimming', 'Wrestling', 'Fencing', 'Athletics', 'Shooting', 'Cycling', 'Rowing', 'Gymnastics', 'Combined']
sport_dropdown = widgets.Dropdown(options=sports, value='Combined', description='Sport')
kde_chkbox = widgets.Checkbox(value=False, description='kde')
display_data_chkbox = widgets.Checkbox(value=False, description='display_data')

widgets.interact(plot_figure, sport=sport_dropdown, kde=kde_chkbox, display_data=display_data_chkbox);

interactive(children=(Dropdown(description='Sport', index=8, options=('Swimming', 'Wrestling', 'Fencing', 'Ath…

# Question 3

In [6]:
data = pd.read_csv('used_cars.csv')
data.head(3)

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,Audi
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,Audi
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,Audi


In [7]:
data = pd.read_csv('used_cars.csv')

year_dropdown = widgets.Dropdown(
    options=data['year'].unique(),
    description='Year'
)

def brand(year):
    brands = data[data['year'] == year]['brand'].unique()
    brand_dropdown.options = brands

brand_dropdown = widgets.Dropdown(
    description='Brand'
)


def model(year, brand):
    models = data[(data['year'] == year) & (data['brand'] == brand)]['model'].unique()
    model_multiselect.options = models

model_multiselect = widgets.SelectMultiple(
    description='Model'
)


def plot_data(year, brand, models):
    selected_data = data[(data['year'] == year) & (data['brand'] == brand) & (data['model'].isin(models))]
    plt.figure(figsize=(6, 4))
    for model in models:
        model_data = selected_data[selected_data['model'] == model]
        sns.scatterplot(x='price', y='mileage', data=model_data, label=model)
    plt.title(f'{year} {brand}')
    plt.xlabel('Price')
    plt.ylabel('Mileage')
    plt.legend(title='model', loc='upper right')
    plt.show()


widgets.interactive(brand, year=year_dropdown)
widgets.interactive(model, year=year_dropdown, brand=brand_dropdown)
widgets.interactive(plot_data, year=year_dropdown, brand=brand_dropdown, models=model_multiselect)

interactive(children=(Dropdown(description='Year', options=(2017, 2016, 2019, 2015, 2014, 2018, 2013, 2020, 20…